<h1><center>NHL DATA SCRAPE AND VISUALIZATION</center></h1>
<h2><center>A Learning Project</center></h2>

The following is an attempt to learn pandas, numpy, and beautifulsoup in order to create a repository of data scrapped from the web, clean it, and then process it to draw and test assumptions against that data. This will utilize the NHL API along with the NHL Play by Play sheet. One of the reasons to use a Jupyter Notebook is to avoid multiple calls to the mentioned websites but still continue to work with the data they provide.

First we import the required python libraries. 
- BeautifulSoup will allow us to manipulate the html files from the NHL Play by Play 
- NHLget is a self made module which accesses the NHL API and the NHL Play by Play pages. It has three functions: One to return a list of games played during the provided time period (NHL API). One to return game information about a specific game (NHL API), and the third to provide play by play information (NHL Play by Play).

In [1]:
import NHLget
from copy import deepcopy

In [2]:
all_plays = NHLget.get_player_on_ice_info('2021020001')

In [3]:
game_json = NHLget.get_game_info('2021020001')

In [4]:
plays = game_json['liveData']['plays']['allPlays']

play_id_dict = {'GAME_SCHEDULED': 'PGSTR', 'PERIOD_READY': 'PGEND', 
'PERIOD_START': 'PSTR', 'FACEOFF': 'FAC', 'HIT': 'HIT', 'STOP': 'STOP', 
'SHOT': 'SHOT', 'TAKEAWAY': 'TAKE', 'BLOCKED_SHOT': 'BLOCK', 
'MISSED_SHOT': 'MISS', 'GIVEAWAY': 'GIVE', 'PERIOD_END': 'PEND', 
'PERIOD_OFFICIAL': '', 
  'GOAL': 'GOAL', 
  'PENALTY': 'PENL', 
  'GAME_END': 'GEND', 
  'GAME_OFFICIAL': ''}


In [5]:
# playId, period, strength, [Time Elapsed, Time Remaining], Event, Description, Away Team Players on Ice, Home Team Players on Ice, players involved, dateTime, coordinates
updated_plays = deepcopy(all_plays)
track_i = 0
for mainPlay in updated_plays:
  if mainPlay[0] == '#':
    continue
  if mainPlay[3][0][0:2] == '0:':
    mainPlay[3][0] = '00:' + mainPlay[3][0][-2:]
  for ind in range(track_i, len(plays)):
    if all([
        play_id_dict[plays[ind]['result']['eventTypeId']] == mainPlay[4],
        plays[ind]['about']['period'] == int(mainPlay[1]),
        plays[ind]['about']['periodTime'] == mainPlay[3][0]
    ]):
      if 'players' in plays[ind]:
        mainPlay.append(plays[ind]['players'][0])
        if len(plays[ind]['players']) > 1:
          mainPlay.append(plays[ind]['players'][1])
        if len(plays[ind]['players']) > 2:
          mainPlay.append(plays[ind]['players'][2])
        if len(plays[ind]['players']) > 3:
          mainPlay.append(plays[ind]['players'][3])
      mainPlay.append(plays[ind]['about']['dateTime'])
      mainPlay.append(plays[ind]['coordinates'])
      track_i = ind + 1

print(all_plays[6])
print(updated_plays[6])

['6', '1', 'EV', ['0:18', '19:42'], 'HIT', 'TBL #18 PALAT HIT PIT #77 CARTER, Off. Zone', {'C': '77', 'R': '17', 'L': '43', 'D1': '8', 'D2': '58', 'G': '35'}, {'C': '21', 'R': '86', 'L': '18', 'D1': '27', 'D2': '81', 'G': '88'}]
['6', '1', 'EV', ['00:18', '19:42'], 'HIT', 'TBL #18 PALAT HIT PIT #77 CARTER, Off. Zone', {'C': '77', 'R': '17', 'L': '43', 'D1': '8', 'D2': '58', 'G': '35'}, {'C': '21', 'R': '86', 'L': '18', 'D1': '27', 'D2': '81', 'G': '88'}, {'player': {'id': 8476292, 'fullName': 'Ondrej Palat', 'link': '/api/v1/people/8476292'}, 'playerType': 'Hitter'}, {'player': {'id': 8470604, 'fullName': 'Jeff Carter', 'link': '/api/v1/people/8470604'}, 'playerType': 'Hittee'}, '2021-10-12T23:44:24Z', {'x': 46.0, 'y': 40.0}]
